<a href="https://colab.research.google.com/github/okana2ki/intro-to-AI/blob/main/timetable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 生成AIを使う上でのセキュリティ上の注意

ChatGPTで時間割データや履修の手引きデータを扱う場合、以下の手順でオプトアウトすると、学習データとして使われなくなり、安全性が増します。他の生成AIを使う場合も同様に学習データとして使われないための手順を実行して下さい。

1. ChatGPTの画面で、自分のアカウント名をクリック
2. 「設定」をクリック
3. 「データコントロール」をクリック
4. 「すべての人のためにモデルを改善する」をオフにする

# 時間割エクセルファイルの読み込みとhtmlファイルへの出力

## GPT o3-mini-highへの指示（例）

添付した時間割データを読み込んで、html形式の表としてhtmlファイルに出力して下さい。ただし、添付した時間割データの形式は以下の通り：

### 各行の内容

以下では、行番号は、ヘッダー行も含めて１行目、２行目、・・・と数える。

・ヘッダー行は３行：１行目は表のタイトル、２行目は曜日のラベル、３行目は記入項目のラベル。データ本体は４行目以降。

- 1限の前期開講科目は、5行から9行に記載
- 1限の通年開講科目は、12行から15行に記載
- 1限の後期開講科目は、18行から22行に記載

- 2限の前期開講科目は、25行から30行に記載
- 2限の通年開講科目は、33行から37行に記載
- 2限の後期開講科目は、40行から46行に記載

- 3限の前期開講科目は、49行から53行に記載
- 3限の通年開講科目は、56行から62行に記載
- 3限の後期開講科目は、65行から69行に記載

- 4限の前期開講科目は、72行から76行に記載
- 4限の通年開講科目は、79行から82行に記載
- 4限の後期開講科目は、85行から89行に記載

- 5限の前期開講科目は、92行から96行に記載
- 5限の通年開講科目は、99行から102行に記載
- 5限の後期開講科目は、105行から109行に記載



### 各列の内容

・A列は空列。B列は時限のラベル（１から５の数字）が記入されており、各時限の開始・終了時刻（9:00, 10:30など）も書かれている。C列は学期のラベル（前期、通年、後期）。データ本体はD列以降。

月曜の科目の記載列は次の通り：
- 年次：D列
- 科目：E列
- コード：H列
- 教員：I列
- 教室：J列

火曜の科目の記載列は次の通り：
- 年次：K列
- 科目：L列
- コード：M列
- 教員：N列
- 教室：O列

水曜の科目の記載列は次の通り：
- 年次：P列
- 科目：Q列
- コード：T列
- 教員：U列
- 教室：V列

木曜の科目の記載列は次の通り：
- 年次：W列
- 科目：X列
- コード：AA列
- 教員：AB列
- 教室：AD列

金曜の科目の記載列は次の通り：
- 年次：AE列
- 科目：AF列
- コード：AI列
- 教員：AJ列
- 教室：AL列

・「科目」列に記載された科目名には表記揺れがあるため、正規化が必要
- すべての空白文字を削除
- 英数字と括弧類を半角に統一（他の括弧類と同様全角〈〉も半角<>に統一）
- 半角カタカナを全角カタカナに統一

・「科目」列には通常科目名だけが記載されているが、以下の科目では、「科目名」に「クラス」が付記されている：
- 情報処理応用<営1>/<営2>/<営3>/<営4>
- 情報処理基礎<営>
- 教育行政学[教職]
- スポーツ総合<営>
- 実践英会話I(1年次対象)
- 実践英会話II(1年次対象)
- 数学ステップアップ講座A(1クラス)/(2クラス)
- 数学ステップアップ講座B(1クラス)/(2クラス)
- 教育原理[教職]
- 教育心理学[教職]
- 教職概論[教職]

・これらの「クラスが科目名に付記された科目」については、「科目」列に記された内容を、項目「科目」と項目「クラス」に分けて内部データ構造に格納し、HTML出力においても、「科目」と「クラス」に分けて表示して下さい。

- 科目：情報処理応用、クラス：営1/営2/営3/営4
- 科目：情報処理基礎、クラス：営
- 科目：教育行政学、クラス：教職
- 科目：スポーツ総合、クラス：営
- 科目：実践英会話I、クラス：1年次対象
- 科目：実践英会話II、クラス：1年次対象
- 科目：数学ステップアップ講座A、クラス：1クラス/2クラス
- 科目：数学ステップアップ講座B、クラス：1クラス/2クラス
- 科目：教育原理、クラス：教職
- 科目：教育心理学、クラス：教職
- 科目：教職概論、クラス：教職

### 例外処理

木曜３限の通年科目については、元データが特殊な形式で書かれているため、例外処理が必要。
このセルの掲載科目は次の７科目として下さい。

- 年次：➊、科目：進路研究演習1、クラス：a 武次玄三、コード：21、教室：5401
- 年次：➊、科目：進路研究演習1、クラス：b 安井　誠、コード：22、教室：5402
- 年次：➊、科目：進路研究演習1、クラス：c 雜賀智子、コード：23、教室：5403
- 年次：➊、科目：進路研究演習1、クラス：d 堀下　翔、コード：24、教室：5404
- 年次：➊、科目：進路研究演習1、クラス：e 岩切道雄、コード：25、教室：5405
- 年次：➊、科目：進路研究演習1、クラス：f 若松朋也、コード：30、教室：3104
- 年次：➊、科目：進路研究演習1、クラス：g 藏屋滉平、コード：31、教室：3203

## 事前準備

ワードファイルの読み込みに必要なライブラリのインストール

In [ ]:
!pip install python-docx # Install the python-docx package, which provides the 'docx' module

In [1]:
# prompt: google driveからファイルを読み込む

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 生成されたプログラム

In [6]:
import pandas as pd
import re
import unicodedata

# ファイルのパスを指定します。
file_path = '/content/drive/MyDrive/Colab_files/2025keiei1.xlsx'  # 自分のファイルのパスに変更してください

# Excelファイルの読み込み（ヘッダーなし）
df = pd.read_excel(file_path, header=None)

# ヘッダー部の取得
title = df.iloc[0, 0]  # 1行目（タイトル）
weekdays = df.iloc[1, 3:3+5].tolist()  # 2行目、例としてD～H列を曜日ラベルと仮定
labels = df.iloc[2, 3:3+5].tolist()  # 3行目、項目ラベル（例）

# 定義：各時限・学期の行番号のブロック情報
# 0-indexedに変換済（例：Excelの5行目はdf.iloc[4]）
blocks = {
    "1限": {"前期": (4, 9), "通年": (11, 15), "後期": (17, 22)},
    "2限": {"前期": (24, 30), "通年": (32, 37), "後期": (39, 46)},
    "3限": {"前期": (48, 53), "通年": (55, 62), "後期": (64, 69)},
    "4限": {"前期": (71, 76), "通年": (78, 82), "後期": (84, 89)},
    "5限": {"前期": (91, 96), "通年": (98, 102), "後期": (104, 109)}
}

# 曜日ごとの列マッピング（0-indexedに変換済）
columns_map = {
    "月": {"年次": 3, "科目": 4, "コード": 7, "教員": 8, "教室": 9},
    "火": {"年次": 10, "科目": 11, "コード": 12, "教員": 13, "教室": 14},
    "水": {"年次": 15, "科目": 16, "コード": 19, "教員": 20, "教室": 21},
    "木": {"年次": 22, "科目": 23, "コード": 26, "教員": 27, "教室": 29},
    "金": {"年次": 30, "科目": 31, "コード": 34, "教員": 35, "教室": 37}
}

def normalize_subject(subject_str):
    """
    科目名の表記揺れを正規化する。
    すべての空白文字を削除し、英数字と括弧類を半角に統一。
    半角カタカナを全角カタカナに統一。
    """
    if pd.isna(subject_str):
        return ""
    # 正規表現を使用して、すべての空白文字（全角スペース、半角スペース、タブ、改行など）を削除
    norm = re.sub(r'\s+', '', subject_str)
    # NFKC正規化：英数字を半角に統一；括弧類を半角に統一；半角カタカナを全角カタカナに統一
    norm = unicodedata.normalize('NFKC', norm)
    # 全角〈〉を半角<>に統一
    norm = norm.replace("〈", "<").replace("〉", ">")
    return norm

# 対象科目一覧（科目名にクラス情報が付記される科目）
subject_keys = [
    "情報処理応用", "情報処理基礎", "教育行政学", "スポーツ総合",
    "実践英会話I", "実践英会話II", "数学ステップアップ講座A", "数学ステップアップ講座B",
    "教育原理", "教育心理学", "教職概論"
]

def split_subject_class(subject):
    """
    対象科目について、科目名とクラス情報を分割して返す。
    対象科目に該当する場合は、先頭部分を科目名、残りをクラス情報として扱う。
    複数のクラスが記載されている場合は、'/' で結合します。
    """
    for key in subject_keys:
        if subject.startswith(key):
            remainder = subject[len(key):]
            if remainder:
                # '/'で分割し、各部分から括弧類 (<,>,(,),[,]) を除去
                parts = remainder.split('/')
                cleaned_parts = []
                for part in parts:
                    part_clean = part.strip('<>()[]')
                    if part_clean:
                        cleaned_parts.append(part_clean)
                class_str = '/'.join(cleaned_parts)
            else:
                class_str = ""
            return key, class_str
    return subject, ""  # 対象外の場合はそのまま返す

# データを格納する構造
timetable = {}

# 各時限・学期ごとにデータを抽出
for period, sem_dict in blocks.items():
    timetable[period] = {}
    for sem, (start, end) in sem_dict.items():
        # 各ブロック内の行を取得
        block_df = df.iloc[start:end]
        # 曜日ごとの情報を格納する辞書
        day_data = {}
        for day, cols in columns_map.items():
            entries = []
            for idx, row in block_df.iterrows():
                # 各項目の値を取得
                year = row.iloc[cols["年次"]]
                subject_raw = normalize_subject(row.iloc[cols["科目"]])
                # 科目・クラスの分割処理を実施
                base_subject, class_info = split_subject_class(subject_raw)
                subject = normalize_subject(row.iloc[cols["科目"]])
                code = row.iloc[cols["コード"]]
                teacher = row.iloc[cols["教員"]]
                room = row.iloc[cols["教室"]]
                # 空でなければ情報をまとめる
                if pd.notna(subject) and subject != "":
                    entry = {
                        "年次": year,
                        "科目": base_subject,
                        "クラス": class_info,
                        "コード": code,
                        "教員": teacher,
                        "教室": room
                    }
                    entries.append(entry)
            day_data[day] = entries
        timetable[period][sem] = day_data

# 例外処理：木曜3限通年のセルの内容を無条件に上書き
if "3限" in timetable and "通年" in timetable["3限"]:
    timetable["3限"]["通年"]["木"] = [
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "a 武次玄三", "コード": 21, "教員": None, "教室": 5401},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "b 安井　誠", "コード": 22, "教員": None, "教室": 5402},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "c 雜賀智子", "コード": 23, "教員": None, "教室": 5403},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "d 堀下　翔", "コード": 24, "教員": None, "教室": 5404},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "e 岩切道雄", "コード": 25, "教員": None, "教室": 5405},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "f 若松朋也", "コード": 30, "教員": None, "教室": 3104},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "g 藏屋滉平", "コード": 31, "教員": None, "教室": 3203},
    ]

In [8]:
# HTMLテーブルの生成
html = []
html.append('<html>')
html.append('<head><meta charset="UTF-8"><style>')
html.append('table { border-collapse: collapse; }')
html.append('th, td { border: 1px solid #000; padding: 5px; }')
html.append('</style></head>')
html.append('<body>')

# タイトル行
html.append(f'<h2>{title}</h2>')

# ヘッダー：左端に「時限」と「学期」の列、続いて曜日列を設定
weekdays_list = ["月", "火", "水", "木", "金"]
html.append('<table>')
html.append('<thead>')
html.append('<tr>')
html.append('<th>時限</th>')
html.append('<th>学期</th>')
for day in weekdays_list:
    html.append(f'<th>{day}</th>')
html.append('</tr>')
html.append('</thead>')
html.append('<tbody>')

# 各時限・学期ごとのデータ出力
for period in ["1限", "2限", "3限", "4限", "5限"]:
    # 3行（前期・通年・後期）のうち最初の行で時限セルを rowspan="3" で出力
    for idx, sem in enumerate(["前期", "通年", "後期"]):
        html.append('<tr>')
        if idx == 0:
            html.append(f'<td rowspan="3">{period}</td>')
        html.append(f'<td>{sem}</td>')
        for day in weekdays_list:
            cell_entries = timetable.get(period, {}).get(sem, {}).get(day, [])
            if cell_entries:
                cell_html = ""
                for entry in cell_entries:
                    parts = []
                    if pd.notna(entry.get("年次")):
                        parts.append(f'年次：{entry.get("年次")}')
                    parts.append(f'科目：{entry.get("科目")}')
                    if entry.get("クラス"):
                        parts.append(f'クラス：{entry.get("クラス")}')
                    if pd.notna(entry.get("コード")):
                        parts.append(f'コード：{entry.get("コード")}')
                    if pd.notna(entry.get("教員")):
                        parts.append(f'教員：{entry.get("教員")}')
                    if pd.notna(entry.get("教室")):
                        parts.append(f'教室：{entry.get("教室")}')
                    cell_html += "<div>" + " | ".join(parts) + "</div>"
                html.append(f'<td>{cell_html}</td>')
            else:
                html.append('<td></td>')
        html.append('</tr>')

html.append('</tbody>')
html.append('</table>')
html.append('</body></html>')

html_output = "\n".join(html)

# HTMLファイルに書き込む
with open('/content/drive/MyDrive/Colab_files/timetable.html', 'w', encoding='utf-8') as file:
    file.write(html_output)

# シラバスへのリンクの追加と、科目名の科目区分ごとの色分け

## gpt o3-mini-highへの指示（例）

添付ファイルは、科目を３種類（総合教育科目／専門教育科目／自由科目）に分け、それぞれの種類ごとに、科目名とシラバスへのリンクを掲載したものです。時間割上の科目名と対応する科目を添付ファイルから見つけ、HTML時間割上の科目名をクリックすると各科目のシラバスを参照できるようにして下さい。
また、HTML時間割上の科目名の文字色を、総合教育科目は赤文字、専門教育科目は黒文字、自由科目は水色文字で表示して下さい。
なお、添付ファイル中の科目名は、時間割上の科目名の正規化処理と同じ処理を施した後で、同一科目を探す必要があります。

## 生成されたプログラム

In [12]:
import pandas as pd
import re
import unicodedata
from docx import Document

# ======================================
# 1. ファイルパス等の設定（必要に応じて変更）
# ======================================
excel_file_path = '/content/drive/MyDrive/Colab_files/2025keiei1.xlsx'
sogo_file = '/content/drive/MyDrive/Colab_files/sogo.docx'
senmon_file = '/content/drive/MyDrive/Colab_files/senmon.docx'
jiyu_file = '/content/drive/MyDrive/Colab_files/jiyu.docx'

# ======================================
# 2. Excelファイルの読み込み
# ======================================
df = pd.read_excel(excel_file_path, header=None)

# ヘッダー部の取得
title = df.iloc[0, 0]  # 1行目：タイトル
weekdays = df.iloc[1, 3:3+5].tolist()  # 2行目（曜日ラベル例）
labels = df.iloc[2, 3:3+5].tolist()      # 3行目（項目ラベル例）

# ======================================
# 3. 各ブロック（時限・学期）の設定（0-indexed）
# ======================================
blocks = {
    "1限": {"前期": (4, 9), "通年": (11, 15), "後期": (17, 22)},
    "2限": {"前期": (24, 30), "通年": (32, 37), "後期": (39, 46)},
    "3限": {"前期": (48, 53), "通年": (55, 62), "後期": (64, 69)},
    "4限": {"前期": (71, 76), "通年": (78, 82), "後期": (84, 89)},
    "5限": {"前期": (91, 96), "通年": (98, 102), "後期": (104, 109)}
}

# ======================================
# 4. 曜日ごとの列マッピング（0-indexed）
# ======================================
columns_map = {
    "月": {"年次": 3, "科目": 4, "コード": 7, "教員": 8, "教室": 9},
    "火": {"年次": 10, "科目": 11, "コード": 12, "教員": 13, "教室": 14},
    "水": {"年次": 15, "科目": 16, "コード": 19, "教員": 20, "教室": 21},
    "木": {"年次": 22, "科目": 23, "コード": 26, "教員": 27, "教室": 29},
    "金": {"年次": 30, "科目": 31, "コード": 34, "教員": 35, "教室": 37}
}

# ======================================
# 5. 科目名の正規化関数
# ======================================
def normalize_subject(subject_str):
    """
    科目名の表記揺れを正規化する。
    すべての空白文字を削除し、英数字や括弧類をNFKC正規化で統一。
    また、全角〈〉は半角<>に置換する。
    """
    if pd.isna(subject_str):
        return ""
    norm = re.sub(r'\s+', '', subject_str)
    norm = unicodedata.normalize('NFKC', norm)
    norm = norm.replace("〈", "<").replace("〉", ">")
    return norm

# ======================================
# 6. 対象科目に対して、科目名とクラスを分割する関数
# ======================================
subject_keys = [
    "情報処理応用", "情報処理基礎", "教育行政学", "スポーツ総合",
    "実践英会話I", "実践英会話II", "数学ステップアップ講座A", "数学ステップアップ講座B",
    "教育原理", "教育心理学", "教職概論"
]

def split_subject_class(subject):
    """
    対象科目の場合、先頭の科目名部分と、残りのクラス情報部分に分割する。
    複数クラスの場合は '/' で結合する。
    """
    for key in subject_keys:
        if subject.startswith(key):
            remainder = subject[len(key):]
            if remainder:
                parts = remainder.split('/')
                cleaned_parts = [part.strip('<>()[]') for part in parts if part.strip('<>()[]')]
                class_str = '/'.join(cleaned_parts)
            else:
                class_str = ""
            return key, class_str
    return subject, ""  # 対象外の場合はそのまま

# ======================================
# 7. docx内のハイパーリンクを抽出して、科目名とシラバスリンクの対応表を作成
# ======================================
def load_syllabus_mapping(docx_file):
    mapping = {}
    doc = Document(docx_file)
    rels = doc.part.rels
    # 各段落内のハイパーリンク要素を抽出
    for para in doc.paragraphs:
        # XPathでw:hyperlink要素を取得
        hyperlink_elements = para._p.xpath('.//w:hyperlink')
        for hyperlink in hyperlink_elements:
            r_id = hyperlink.get('{http://schemas.openxmlformats.org/officeDocument/2006/relationships}id')
            if r_id and r_id in rels:
                # ハイパーリンク内のテキストを取得（w:t要素）
                texts = [node.text for node in hyperlink.xpath('.//w:t') if node.text]
                text = ''.join(texts)
                norm_text = normalize_subject(text)
                if norm_text:
                    url = rels[r_id].target_ref
                    mapping[norm_text] = url
    return mapping

sogo_mapping = load_syllabus_mapping(sogo_file)
senmon_mapping = load_syllabus_mapping(senmon_file)
jiyu_mapping = load_syllabus_mapping(jiyu_file)

# ======================================
# 8. 時間割データの抽出（各時限・学期・曜日ごと）
# ======================================
timetable = {}
for period, sem_dict in blocks.items():
    timetable[period] = {}
    for sem, (start, end) in sem_dict.items():
        block_df = df.iloc[start:end]
        day_data = {}
        for day, cols in columns_map.items():
            entries = []
            for idx, row in block_df.iterrows():
                year = row.iloc[cols["年次"]]
                subject_raw = normalize_subject(row.iloc[cols["科目"]])
                base_subject, class_info = split_subject_class(subject_raw)
                code = row.iloc[cols["コード"]]
                teacher = row.iloc[cols["教員"]]
                room = row.iloc[cols["教室"]]
                if pd.notna(subject_raw) and subject_raw != "":
                    entry = {
                        "年次": year,
                        "科目": base_subject,
                        "クラス": class_info,
                        "コード": code,
                        "教員": teacher,
                        "教室": room
                    }
                    entries.append(entry)
            day_data[day] = entries
        timetable[period][sem] = day_data

# ======================================
# 9. 例外処理：木曜3限通年のセルを無条件に上書き
# ======================================
if "3限" in timetable and "通年" in timetable["3限"]:
    timetable["3限"]["通年"]["木"] = [
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "a 武次玄三", "コード": 21, "教員": None, "教室": 5401},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "b 安井　誠", "コード": 22, "教員": None, "教室": 5402},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "c 雜賀智子", "コード": 23, "教員": None, "教室": 5403},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "d 堀下　翔", "コード": 24, "教員": None, "教室": 5404},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "e 岩切道雄", "コード": 25, "教員": None, "教室": 5405},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "f 若松朋也", "コード": 30, "教員": None, "教室": 3104},
        {"年次": "➊", "科目": "進路研究演習1", "クラス": "g 藏屋滉平", "コード": 31, "教員": None, "教室": 3203},
    ]

# ======================================
# 10. HTMLテーブルの生成（左側に「時限」＋「学期」列追加・時限セルは3行結合）
# ======================================
html = []
html.append('<html>')
html.append('<head><meta charset="UTF-8"><style>')
html.append('table { border-collapse: collapse; }')
html.append('th, td { border: 1px solid #000; padding: 5px; }')
html.append('</style></head>')
html.append('<body>')

# タイトル
html.append(f'<h2>{title}</h2>')

# ヘッダー（左に「時限」「学期」列、続いて曜日列）
weekdays_list = ["月", "火", "水", "木", "金"]
html.append('<table>')
html.append('<thead>')
html.append('<tr>')
html.append('<th>時限</th>')
html.append('<th>学期</th>')
for day in weekdays_list:
    html.append(f'<th>{day}</th>')
html.append('</tr>')
html.append('</thead>')
html.append('<tbody>')

# 各時限・学期ごとのデータ出力
for period in ["1限", "2限", "3限", "4限", "5限"]:
    for idx, sem in enumerate(["前期", "通年", "後期"]):
        html.append('<tr>')
        if idx == 0:
            html.append(f'<td rowspan="3">{period}</td>')
        html.append(f'<td>{sem}</td>')
        for day in weekdays_list:
            cell_entries = timetable.get(period, {}).get(sem, {}).get(day, [])
            if cell_entries:
                cell_html = ""
                for entry in cell_entries:
                    parts = []
                    if pd.notna(entry.get("年次")):
                        parts.append(f'年次：{entry.get("年次")}')

                    # 科目表示：科目名に対してシラバスリンクと文字色を適用
                    subj = entry.get("科目")
                    link = None
                    color = None
                    if subj in sogo_mapping:
                        link = sogo_mapping[subj]
                        color = "red"
                    elif subj in senmon_mapping:
                        link = senmon_mapping[subj]
                        color = "black"
                    elif subj in jiyu_mapping:
                        link = jiyu_mapping[subj]
                        color = "#00FFFF"  # 水色
                    if link:
                        subj_html = f'<a href="{link}" style="color:{color}; text-decoration:none;">{subj}</a>'
                    else:
                        subj_html = subj
                    parts.append(f'科目：{subj_html}')

                    if entry.get("クラス"):
                        parts.append(f'クラス：{entry.get("クラス")}')
                    if pd.notna(entry.get("コード")):
                        parts.append(f'コード：{entry.get("コード")}')
                    if pd.notna(entry.get("教員")):
                        parts.append(f'教員：{entry.get("教員")}')
                    if pd.notna(entry.get("教室")):
                        parts.append(f'教室：{entry.get("教室")}')

                    cell_html += "<div>" + " | ".join(parts) + "</div>"
                html.append(f'<td>{cell_html}</td>')
            else:
                html.append('<td></td>')
        html.append('</tr>')

html.append('</tbody>')
html.append('</table>')
html.append('</body></html>')

html_output = "\n".join(html)

# HTMLファイルに書き込む
with open('/content/drive/MyDrive/Colab_files/timetable.html', 'w', encoding='utf-8') as file:
    file.write(html_output)